Copyright (c) Microsoft Corporation. All rights reserved.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/tutorials/regression-part2-automated-ml.png)

# Tutorial: Use automated machine learning to predict taxi fares

In this tutorial, you use automated machine learning in Azure Machine Learning service to create a regression model to predict NYC taxi fare prices. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

In this tutorial you learn the following tasks:

* Download, transform, and clean data using Azure Open Datasets
* Train an automated machine learning regression model
* Calculate model accuracy

If you donâ€™t have an Azure subscription, create a free account before you begin. Try the [free or paid version](https://aka.ms/AMLFree) of Azure Machine Learning service today.

## Prerequisites

* Complete the [setup tutorial](https://docs.microsoft.com/azure/machine-learning/service/tutorial-1st-experiment-sdk-setup) if you don't already have an Azure Machine Learning service workspace or notebook virtual machine.
* After you complete the setup tutorial, open the **tutorials/regression-automated-ml.ipynb** notebook using the same notebook server.

This tutorial is also available on [GitHub](https://github.com/Azure/MachineLearningNotebooks/tree/master/tutorials) if you wish to run it in your own [local environment](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-environment#local). Run `pip install azureml-sdk[automl] azureml-opendatasets azureml-widgets` to get the required packages.

## Download and prepare data

Import the necessary packages. The Open Datasets package contains a class representing each data source (`NycTlcGreen` for example) to easily filter date parameters before downloading.

In [4]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid `MemoryError` with large datasets. To download taxi data, iteratively fetch one month at a time, and before appending it to `green_taxi_df` randomly sample 2,000 records from each month to avoid bloating the dataframe. Then preview the data.

In [5]:
green_taxi_df = pd.DataFrame([])
start = datetime.strptime("1/1/2015","%m/%d/%Y")
end = datetime.strptime("1/31/2015","%m/%d/%Y")

for sample_month in range(12):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .get_tabular_dataset().to_pandas_dataframe()
    green_taxi_df = green_taxi_df.append(temp_df_green.sample(2000))
    
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
2015-01-23 23:39:27,1,2015-01-23 23:39:27,2015-01-23 23:52:34,1,5.40,None,None,-73.957947,40.721958,-74.001305,...,1,17.0,0.5,0.5,0.3,2.00,0.0,None,20.30,1.0
2015-01-04 01:47:10,1,2015-01-04 01:47:10,2015-01-04 02:01:46,1,3.30,None,None,-73.945259,40.808632,-73.947472,...,1,13.5,0.5,0.5,0.3,2.95,0.0,None,17.75,1.0
2015-01-13 10:42:44,2,2015-01-13 10:42:44,2015-01-13 10:52:04,6,1.43,None,None,-73.994698,40.684830,-73.971115,...,2,8.0,0.0,0.5,0.3,0.00,0.0,None,8.80,1.0
2015-01-24 03:18:26,1,2015-01-24 03:18:26,2015-01-24 03:39:05,1,3.10,None,None,-73.988228,40.685822,-73.941391,...,2,14.5,0.5,0.5,0.3,0.00,0.0,None,15.80,1.0
2015-01-19 07:39:28,1,2015-01-19 07:39:28,2015-01-19 07:41:08,1,0.30,None,None,-73.926392,40.833191,-73.928108,...,2,0.0,0.0,0.0,0,0.00,0.0,None,0.00,2.0
2015-01-21 17:09:17,2,2015-01-21 17:09:17,2015-01-21 17:30:40,1,6.77,None,None,-73.937744,40.848373,-73.836502,...,1,22.0,1.0,0.5,0.3,6.90,0.0,None,30.70,1.0
2015-01-15 21:14:34,1,2015-01-15 21:14:34,2015-01-15 21:55:32,1,9.70,None,None,-73.951958,40.789955,-73.848396,...,2,35.5,0.5,0.5,0.3,0.00,0.0,None,36.80,1.0
2015-01-20 14:37:07,2,2015-01-20 14:37:07,2015-01-20 14:47:39,5,2.08,None,None,-73.846764,40.723057,-73.835960,...,2,9.5,0.0,0.5,0.3,0.00,0.0,None,10.30,1.0
2015-01-11 13:18:55,2,2015-01-11 13:18:55,2015-01-12 12:57:27,1,7.53,None,None,-73.992111,40.699314,-73.973488,...,2,29.0,0.0,0.5,0.3,0.00,0.0,None,29.80,1.0
2015-01-26 09:18:34,2,2015-01-26 09:18:34,2015-01-26 09:51:21,1,6.60,None,None,-73.975372,40.685837,-73.996399,...,1,26.0,0.0,0.5,0.3,0.00,0.0,None,26.80,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day, and will allow the model to factor in time-based seasonality. 

Use the `apply()` function on the dataframe to iteratively apply the `build_time_features()` function to each row in the taxi data.

In [6]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df.head(10)

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,month_num,day_of_month,day_of_week,hour_of_day
2015-01-23 23:39:27,1,2015-01-23 23:39:27,2015-01-23 23:52:34,1,5.40,None,None,-73.957947,40.721958,-74.001305,...,0.3,2.00,0.0,None,20.30,1.0,1,23,4,23
2015-01-04 01:47:10,1,2015-01-04 01:47:10,2015-01-04 02:01:46,1,3.30,None,None,-73.945259,40.808632,-73.947472,...,0.3,2.95,0.0,None,17.75,1.0,1,4,6,1
2015-01-13 10:42:44,2,2015-01-13 10:42:44,2015-01-13 10:52:04,6,1.43,None,None,-73.994698,40.684830,-73.971115,...,0.3,0.00,0.0,None,8.80,1.0,1,13,1,10
2015-01-24 03:18:26,1,2015-01-24 03:18:26,2015-01-24 03:39:05,1,3.10,None,None,-73.988228,40.685822,-73.941391,...,0.3,0.00,0.0,None,15.80,1.0,1,24,5,3
2015-01-19 07:39:28,1,2015-01-19 07:39:28,2015-01-19 07:41:08,1,0.30,None,None,-73.926392,40.833191,-73.928108,...,0,0.00,0.0,None,0.00,2.0,1,19,0,7
2015-01-21 17:09:17,2,2015-01-21 17:09:17,2015-01-21 17:30:40,1,6.77,None,None,-73.937744,40.848373,-73.836502,...,0.3,6.90,0.0,None,30.70,1.0,1,21,2,17
2015-01-15 21:14:34,1,2015-01-15 21:14:34,2015-01-15 21:55:32,1,9.70,None,None,-73.951958,40.789955,-73.848396,...,0.3,0.00,0.0,None,36.80,1.0,1,15,3,21
2015-01-20 14:37:07,2,2015-01-20 14:37:07,2015-01-20 14:47:39,5,2.08,None,None,-73.846764,40.723057,-73.835960,...,0.3,0.00,0.0,None,10.30,1.0,1,20,1,14
2015-01-11 13:18:55,2,2015-01-11 13:18:55,2015-01-12 12:57:27,1,7.53,None,None,-73.992111,40.699314,-73.973488,...,0.3,0.00,0.0,None,29.80,1.0,1,11,6,13
2015-01-26 09:18:34,2,2015-01-26 09:18:34,2015-01-26 09:51:21,1,6.60,None,None,-73.975372,40.685837,-73.996399,...,0.3,0.00,0.0,None,26.80,1.0,1,26,0,9


Remove some of the columns that you won't need for training or additional feature building.

In [7]:
columns_to_remove = ["lpepPickupDatetime", "lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"
                    ]
for col in columns_to_remove:
    green_taxi_df.pop(col)
    
green_taxi_df.head(5)

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
2015-01-23 23:39:27,1,1,5.40,-73.957947,40.721958,-74.001305,40.684811,20.30,1,23,4,23
2015-01-04 01:47:10,1,1,3.30,-73.945259,40.808632,-73.947472,40.775497,17.75,1,4,6,1
2015-01-13 10:42:44,2,6,1.43,-73.994698,40.684830,-73.971115,40.672855,8.80,1,13,1,10
2015-01-24 03:18:26,1,1,3.10,-73.988228,40.685822,-73.941391,40.684284,15.80,1,24,5,3
2015-01-19 07:39:28,1,1,0.30,-73.926392,40.833191,-73.928108,40.830200,0.00,1,19,0,7


### Cleanse data 

Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [8]:
green_taxi_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.775833,1.370042,2.872523,-73.815303,40.682814,-73.842071,40.696315,14.746325,6.500000,15.136583,3.273708,13.603750
std,0.417041,1.044863,2.935639,2.978368,1.642371,2.612882,1.440954,11.876950,3.452124,8.440921,1.955280,6.811253
min,1.000000,0.000000,0.000000,-74.295525,0.000000,-74.335442,0.000000,-180.000000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.060000,-73.959450,40.697185,-73.967569,40.699916,8.000000,3.750000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.910000,-73.945095,40.746758,-73.944584,40.747709,11.300000,6.500000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.640000,-73.917494,40.803277,-73.909531,40.792311,17.800000,9.250000,22.000000,5.000000,19.000000
max,2.000000,9.000000,58.900000,0.000000,40.956421,0.000000,41.156689,533.240000,12.000000,30.000000,6.000000,23.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the bounds of the Manhattan area. This will filter out longer taxi trips or trips that are outliers in respect to their relationship with other features. 

Additionally filter the `tripDistance` field to be greater than zero but less than 31 miles (the haversine distance between the two lat/long pairs). This eliminates long outlier trips that have inconsistent trip cost.

Lastly, the `totalAmount` field has negative values for the taxi fares, which don't make sense in the context of our model, and the `passengerCount` field has bad data with the minimum values being zero.

Filter out these anomalies using query functions, and then remove the last few columns unnecessary for training.

In [9]:
final_df = green_taxi_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88")
final_df = final_df.query("pickupLongitude>=-74.09 and pickupLongitude<=-73.72")
final_df = final_df.query("tripDistance>=0.25 and tripDistance<31")
final_df = final_df.query("passengerCount>0 and totalAmount>0")

columns_to_remove_for_training = ["pickupLongitude", "pickupLatitude", "dropoffLongitude", "dropoffLatitude"]
for col in columns_to_remove_for_training:
    final_df.pop(col)

Call `describe()` again on the data to ensure cleansing worked as expected. You now have a prepared and cleansed set of taxi, holiday, and weather data to use for machine learning model training.

In [10]:
final_df.describe()

,vendorID,passengerCount,tripDistance,totalAmount,month_num,day_of_month,day_of_week,hour_of_day
count,23196.000000,23196.000000,23196.000000,23196.00000,23196.000000,23196.000000,23196.000000,23196.000000
mean,1.777807,1.371012,2.927330,14.79066,6.502199,15.149767,3.280523,13.618426
std,0.415729,1.046557,2.829194,10.78378,3.454498,8.439597,1.954975,6.815284
min,1.000000,1.000000,0.250000,0.05000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.100000,8.30000,3.000000,8.000000,2.000000,9.000000
50%,2.000000,1.000000,1.995000,11.75000,7.000000,15.000000,3.000000,15.000000
75%,2.000000,1.000000,3.700000,17.80000,10.000000,22.000000,5.000000,19.000000
max,2.000000,8.000000,30.820000,533.24000,12.000000,30.000000,6.000000,23.000000


## Configure workspace


Create a workspace object from the existing workspace. A [Workspace](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py) is a class that accepts your Azure subscription and resource information. It also creates a cloud resource to monitor and track your model runs. `Workspace.from_config()` reads the file **config.json** and loads the authentication details into an object named `ws`. `ws` is used throughout the rest of the code in this tutorial.

In [11]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

## Split the data into train and test sets

Split the data into training and test sets by using the `train_test_split` function in the `scikit-learn` library. This function segregates the data into the x (**features**) data set for model training and the y (**values to predict**) data set for testing. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random generator, so that your train-test splits are deterministic.

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(final_df, test_size=0.2, random_state=223)

The purpose of this step is to have data points to test the finished model that haven't been used to train the model, in order to measure true accuracy. 

In other words, a well-trained model should be able to accurately make predictions from data it hasn't already seen. You now have data prepared for auto-training a machine learning model.

## Automatically train a model

To automatically train a model, take the following steps:
1. Define settings for the experiment run. Attach your training data to the configuration, and modify settings that control the training process.
1. Submit the experiment for model tuning. After submitting the experiment, the process iterates through different machine learning algorithms and hyperparameter settings, adhering to your defined constraints. It chooses the best-fit model by optimizing an accuracy metric.

### Define training settings

Define the experiment parameter and model settings for training. View the full list of [settings](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train). Submitting the experiment with these default settings will take approximately 20 minutes, but if you want a shorter run time, reduce the `experiment_timeout_hours` parameter.


|Property| Value in this tutorial |Description|
|----|----|---|
|**iteration_timeout_minutes**|2|Time limit in minutes for each iteration. Reduce this value to decrease total runtime.|
|**experiment_timeout_hours**|0.3|Maximum amount of time in hours that all iterations combined can take before the experiment terminates.|
|**enable_early_stopping**|True|Flag to enable early termination if the score is not improving in the short term.|
|**primary_metric**| spearman_correlation | Metric that you want to optimize. The best-fit model will be chosen based on this metric.|
|**featurization**| auto | By using auto, the experiment can preprocess the input data (handling missing data, converting text to numeric, etc.)|
|**verbosity**| logging.INFO | Controls the level of logging.|
|**n_cross_validations**|5|Number of cross-validation splits to perform when validation data is not specified.|

In [13]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_hours": 0.3,
    "enable_early_stopping": True,
    "primary_metric": 'spearman_correlation',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

Use your defined training settings as a `**kwargs` parameter to an `AutoMLConfig` object. Additionally, specify your training data and the type of model, which is `regression` in this case.

In [14]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='regression',
                             debug_log='automated_ml_errors.log',
                             training_data=x_train,
                             label_column_name="totalAmount",
                             **automl_settings)

Automated machine learning pre-processing steps (feature normalization, handling missing data, converting text to numeric, etc.) become part of the underlying model. When using the model for predictions, the same pre-processing steps applied during training are applied to your input data automatically.

### Train the automatic regression model

Create an experiment object in your workspace. An experiment acts as a container for your individual runs. Pass the defined `automl_config` object to the experiment, and set the output to `True` to view progress during the run. 

After starting the experiment, the output shown updates live as the experiment runs. For each iteration, you see the model type, the run duration, and the training accuracy. The field `BEST` tracks the best running training score based on your metric type.

In [18]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

Running on local machine


ConfigException: ConfigException:
	Message: Module &#39;azureml.train.automl.runtime&#39; is required in the current environment to run local in process runs. Please install this dependency (e.g. `pip install azureml.train.automl.runtime`) or provide a RunConfiguration.
	InnerException: None
	ErrorResponse 
{
    &quot;error&quot;: {
        &quot;code&quot;: &quot;UserError&quot;,
        &quot;message&quot;: &quot;Module &#39;azureml.train.automl.runtime&#39; is required in the current environment to run local in process runs. Please install this dependency (e.g. `pip install azureml.train.automl.runtime`) or provide a RunConfiguration.&quot;,
        &quot;target&quot;: &quot;local&quot;,
        &quot;inner_error&quot;: {
            &quot;code&quot;: &quot;NotSupported&quot;,
            &quot;inner_error&quot;: {
                &quot;code&quot;: &quot;IncompatibleOrMissingDependency&quot;
            }
        }
    }
}

## Explore the results

Explore the results of automatic training with a [Jupyter widget](https://docs.microsoft.com/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py). The widget allows you to see a graph and table of all individual run iterations, along with training accuracy metrics and metadata. Additionally, you can filter on different accuracy metrics than your primary metric with the dropdown selector.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

### Retrieve the best model

Select the best model from your iterations. The `get_output` function returns the best run and the fitted model for the last fit invocation. By using the overloads on `get_output`, you can retrieve the best run and fitted model for any logged metric or a particular iteration.

In [ ]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

### Test the best model accuracy

Use the best model to run predictions on the test data set to predict taxi fares. The function `predict` uses the best model and predicts the values of y, **trip cost**, from the `x_test` data set. Print the first 10 predicted cost values from `y_predict`.

In [ ]:
y_test = x_test.pop("totalAmount")

y_predict = fitted_model.predict(x_test)
print(y_predict[:10])

Calculate the `root mean squared error` of the results. Convert the `y_test` dataframe to a list to compare to the predicted values. The function `mean_squared_error` takes two arrays of values and calculates the average squared error between them. Taking the square root of the result gives an error in the same units as the y variable, **cost**. It indicates roughly how far the taxi fare predictions are from the actual fares.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

Run the following code to calculate mean absolute percent error (MAPE) by using the full `y_actual` and `y_predict` data sets. This metric calculates an absolute difference between each predicted and actual value and sums all the differences. Then it expresses that sum as a percent of the total of the actual values.

In [ ]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

From the two prediction accuracy metrics, you see that the model is fairly good at predicting taxi fares from the data set's features, typically within +- $4.00, and approximately 15% error. 

The traditional machine learning model development process is highly resource-intensive, and requires significant domain knowledge and time investment to run and compare the results of dozens of models. Using automated machine learning is a great way to rapidly test many different models for your scenario.

## Clean up resources

Do not complete this section if you plan on running other Azure Machine Learning service tutorials.

### Stop the notebook VM

If you used a cloud notebook server, stop the VM when you are not using it to reduce cost.

1. In your workspace, select **Compute**.
1. Select the **Notebook VMs** tab in the compute page.
1. From the list, select the VM.
1. Select **Stop**.
1. When you're ready to use the server again, select **Start**.

### Delete everything

If you don't plan to use the resources you created, delete them, so you don't incur any charges.

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next steps

In this automated machine learning tutorial, you did the following tasks:

> * Configured a workspace and prepared data for an experiment.
> * Trained by using an automated regression model locally with custom parameters.
> * Explored and reviewed training results.

[Deploy your model](https://docs.microsoft.com/azure/machine-learning/service/tutorial-deploy-models-with-aml) with Azure Machine Learning service.